<a href="https://colab.research.google.com/github/nschultze/CS577Project/blob/main/ViT_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate -U
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00


Import Data

In [2]:
val_path = '/content/drive/MyDrive/Wild_fire_dataset/val'
train_path = '/content/drive/MyDrive/Wild_fire_dataset/train'
test_path = '/content/drive/MyDrive/Wild_fire_dataset/test'

In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda



Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/train/nofire :
['52293279724_c5250f6aa9_o.png', '23862743953_9107eb402a_o.png']

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/test/nofire :
[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/val/nofire :
[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/train/fire :
['51299053281_e87ae6ec37_o.png', '52464696672_7224ba08b9_o.png', '52464402647_50615af1d0_o.png', '52465662020_3fb295fc5a_o.png']

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/test/fire :
['28347651877_ce21ed134e_o.png', '52465192596_bdd7f52f1c_o.png']

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/val/fire :
['52292032847_11ca7d4982_o.png', '52214758010_97f080bc0c_o.png', '52295603790_d2d6a165fc_o.png', '52465478514_e0dd0f619c_o.png']

Total of 12 . List of all non-RGB images: ['52293279724_c5250f6aa9_o.png', '23862743953_9107eb402a_o.png', '51299053281_e87ae6ec37_o.png', '52464696672_7224ba08b9_o.png', '52464402647_50615af1d0_o.png', '52465662020_3fb295fc5a_o.png', '28347651877_ce21ed134e_o.png', '52465192596_bdd7f52f1c_o.png', '52292032847_11ca7d4982_o.png', '52214758010_97f080bc0c_o.png', '52295603790_d2d6a165fc_o.png', '52465478514_e0dd0f619c_o.png']

In [4]:
from PIL import Image
import os

### a function to return a list of images not in RGB format to filter them out
    ### to adhere to ViT transforms and expectations of data structure
def check_alpha_channel(image_folder):
    print('\nNon-RGB images for ', image_folder, ":" )
    problematic_images = []
    for filename in os.listdir(image_folder):
        image_path = os.path.join(image_folder, filename)
        img = Image.open(image_path)
        if img.mode != 'RGB':
            problematic_images.append(filename)
    print(problematic_images)
    return problematic_images


folder_paths = ['/content/drive/MyDrive/Wild_fire_dataset/train/nofire', '/content/drive/MyDrive/Wild_fire_dataset/test/nofire',
                '/content/drive/MyDrive/Wild_fire_dataset/val/nofire','/content/drive/MyDrive/Wild_fire_dataset/train/fire',
                '/content/drive/MyDrive/Wild_fire_dataset/test/fire', '/content/drive/MyDrive/Wild_fire_dataset/val/fire']
images_to_remove = []
for f in folder_paths:
  for img in check_alpha_channel(f):
    images_to_remove.append(img)

print('\nTotal of', len(images_to_remove), '. List of all non-RGB images:', images_to_remove)



Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/train/nofire :


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/test/nofire :


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/val/nofire :
[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/train/fire :
[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/test/fire :
[]

Non-RGB images for  /content/drive/MyDrive/Wild_fire_dataset/val/fire :
[]

Total of 0 . List of all non-RGB images: []


In [5]:
from datasets import load_dataset

DATASET_DIR = '/content/drive/MyDrive/Wild_fire_dataset'
dataset = load_dataset(name="wildfire", path=DATASET_DIR, data_files={"train": '/content/drive/MyDrive/Wild_fire_dataset/train/**',
                                                                      "test": '/content/drive/MyDrive/Wild_fire_dataset/test/**',
                                                                    'val': '/content/drive/MyDrive/Wild_fire_dataset/val/**'})

Resolving data files:   0%|          | 0/1881 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/408 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/398 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

ViT model card: https://huggingface.co/google/vit-base-patch16-224

In [6]:
from transformers import ViTImageProcessor

## load the processor to transform images to be read by ViT
model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_name_or_path)
print(processor)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



Define transform function

In [7]:
def transform(example_batch):
    inputs = processor([x for x in example_batch['image']], return_tensors='pt')

    inputs['label'] = example_batch['label']
    return inputs

In [8]:
### transform the dataset so it is in format prepared to be read by ViT
prepared_ds = dataset.with_transform(transform)

Define Collate Function for unpacking data and training

In [9]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

Define evaluation metric

Here we just use accuracy

In [10]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

<ipython-input-10-3044fb6e3895>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics_all(p):
    logits, labels = p.predictions, p.label_ids
    predictions = logits.argmax(axis=-1)

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


Import ViT pretrained and translate binary 0/1 labels to text 'fire' 'nofire' for readability later

In [12]:
from transformers import ViTForImageClassification

labels = dataset['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
).to(device)

print(model)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

Set training arguments

In [13]:
from transformers import TrainingArguments
import accelerate

training_args = TrainingArguments(
  output_dir="./vit-base-wildfire",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=2,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)


Create trainer object and pass prepared val and training datasets

In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics_all,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["val"],
    tokenizer=processor,
)

Train the model

**fixed ** Currently there is a bug where it seems some image has more than 3 channels which is impacting the transform's ability to normalize the images

In [16]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.130800,0.348771,0.891960,0.904322,0.891960,0.887977
200,0.102500,0.196399,0.932161,0.932636,0.932161,0.931629


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


***** train metrics *****
  epoch                    =         2.0
  total_flos               = 271503687GF
  train_loss               =      0.2042
  train_runtime            =  0:27:15.15
  train_samples_per_second =       2.301
  train_steps_per_second   =       0.144


Print out performance and metrics on validation set

In [17]:
metrics = trainer.evaluate(prepared_ds['val'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        2.0
  eval_accuracy           =     0.9322
  eval_f1                 =     0.9316
  eval_loss               =     0.1964
  eval_precision          =     0.9326
  eval_recall             =     0.9322
  eval_runtime            = 0:01:59.04
  eval_samples_per_second =      3.343
  eval_steps_per_second   =       0.42


In [18]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


***** test metrics *****
  epoch                   =        2.0
  eval_accuracy           =     0.9755
  eval_f1                 =     0.9754
  eval_loss               =     0.0788
  eval_precision          =     0.9756
  eval_recall             =     0.9755
  eval_runtime            = 0:03:03.54
  eval_samples_per_second =      2.223
  eval_steps_per_second   =      0.278


# Fine tuning

We explore using higher resolution images 384x384

In [19]:
model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor_384 = ViTImageProcessor.from_pretrained(model_name_or_path, size = {"height": 384, "width": 384})

print(processor_384)

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 384,
    "width": 384
  }
}



In [73]:
labels = dataset['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
).to(device)

print(model)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [78]:
from transformers import TrainingArguments
import accelerate

training_args = TrainingArguments(
  output_dir="./vit-base-wildfire-384_4epochs",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=50,
  eval_steps=50,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)


In [79]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics_all,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["val"],
    tokenizer=processor,
)

In [80]:
train_results = trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,0.311200,0.239414,0.896985,0.897887,0.896985,0.897290
100,0.255600,0.399261,0.836683,0.867690,0.836683,0.824606
150,0.126700,0.263461,0.914573,0.918694,0.914573,0.912898
200,0.138600,0.203880,0.924623,0.924817,0.924623,0.924092
250,0.073300,0.198863,0.927136,0.928317,0.927136,0.927430
300,0.040300,0.224413,0.934673,0.934598,0.934673,0.934409
350,0.034000,0.224931,0.944724,0.944724,0.944724,0.944724


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb D

OSError: ignored

In [82]:
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =         2.0
  total_flos               = 271503687GF
  train_loss               =       0.234
  train_runtime            =  0:24:49.52
  train_samples_per_second =       2.526
  train_steps_per_second   =       0.158


In [83]:
metrics = trainer.evaluate(prepared_ds['val'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,0.311200,0.239414,0.896985,0.897887,0.896985,0.897290
100,0.255600,0.399261,0.836683,0.867690,0.836683,0.824606
150,0.126700,0.263461,0.914573,0.918694,0.914573,0.912898
200,0.138600,0.203880,0.924623,0.924817,0.924623,0.924092
250,0.073300,0.198863,0.927136,0.928317,0.927136,0.927430
300,0.040300,0.224413,0.934673,0.934598,0.934673,0.934409
350,0.034000,0.224931,0.944724,0.944724,0.944724,0.944724
400,0.004000,0.237934,0.947236,0.947144,0.947236,0.947133


***** eval metrics *****
  eval_accuracy  = 0.9472
  eval_f1        = 0.9471
  eval_loss      = 0.2379
  eval_precision = 0.9471
  eval_recall    = 0.9472


In [84]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,0.311200,0.239414,0.896985,0.897887,0.896985,0.897290
100,0.255600,0.399261,0.836683,0.867690,0.836683,0.824606
150,0.126700,0.263461,0.914573,0.918694,0.914573,0.912898
200,0.138600,0.203880,0.924623,0.924817,0.924623,0.924092
250,0.073300,0.198863,0.927136,0.928317,0.927136,0.927430
300,0.040300,0.224413,0.934673,0.934598,0.934673,0.934409
350,0.034000,0.224931,0.944724,0.944724,0.944724,0.944724
400,0.004000,0.111420,0.970588,0.970917,0.970588,0.970655


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


***** test metrics *****
  eval_accuracy  = 0.9706
  eval_f1        = 0.9707
  eval_loss      = 0.1114
  eval_precision = 0.9709
  eval_recall    = 0.9706


384x384 image rescale worked better. Let us try to train it for more epochs with more frequent validation set evaluation, warmup and weight decay

In [46]:
model_3 = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
).to(device)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [68]:
training_args_3 = TrainingArguments(
  output_dir="./vit-base-wildfire-384_decay",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=50,
  eval_steps=50,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True, weight_decay = -.8,
)

In [69]:
trainer_3 = Trainer(
    model=model_3,
    args=training_args_3,
    data_collator=collate_fn,
    compute_metrics=compute_metrics_all,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["val"],
    tokenizer=processor,
)

In [70]:
train_results_3 = trainer_3.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,0.279300,0.431364,0.884422,0.887884,0.884422,0.881918
100,0.107500,0.236140,0.914573,0.914573,0.914573,0.914573
150,0.082900,0.214736,0.937186,0.937082,0.937186,0.936977
200,0.058500,0.242480,0.932161,0.932008,0.932161,0.932028
250,0.019300,0.229310,0.929648,0.930705,0.929648,0.928923
300,0.059100,0.229703,0.929648,0.930705,0.929648,0.928923
350,0.008200,0.222765,0.939698,0.942235,0.939698,0.940090
400,0.003500,0.243997,0.939698,0.939848,0.939698,0.939366
450,0.003300,0.253604,0.939698,0.939848,0.939698,0.939366


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DO

***** train metrics *****
  epoch                    =         2.0
  total_flos               = 271503687GF
  train_loss               =       0.234
  train_runtime            =  0:24:49.52
  train_samples_per_second =       2.526
  train_steps_per_second   =       0.158


In [72]:
trainer_3.save_model()
trainer_3.log_metrics("train", train_results.metrics)
trainer_3.save_metrics("train", train_results.metrics)
trainer_3.save_state()

***** train metrics *****
  epoch                    =         2.0
  total_flos               = 271503687GF
  train_loss               =       0.234
  train_runtime            =  0:24:49.52
  train_samples_per_second =       2.526
  train_steps_per_second   =       0.158


In [71]:
metrics_3 = trainer_3.evaluate(prepared_ds['val'])
trainer_3.log_metrics("eval", metrics)
trainer_3.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        2.0
  eval_accuracy           =     0.9422
  eval_f1                 =     0.9416
  eval_loss               =     0.1689
  eval_precision          =     0.9439
  eval_recall             =     0.9422
  eval_runtime            = 0:01:58.97
  eval_samples_per_second =      3.345
  eval_steps_per_second   =       0.42


# Best model performance on test set

Best model has .9447 F1 score on the validation set and uses 4 epochs with image size of 384.

In [85]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,0.311200,0.239414,0.896985,0.897887,0.896985,0.897290
100,0.255600,0.399261,0.836683,0.867690,0.836683,0.824606
150,0.126700,0.263461,0.914573,0.918694,0.914573,0.912898
200,0.138600,0.203880,0.924623,0.924817,0.924623,0.924092
250,0.073300,0.198863,0.927136,0.928317,0.927136,0.927430
300,0.040300,0.224413,0.934673,0.934598,0.934673,0.934409
350,0.034000,0.224931,0.944724,0.944724,0.944724,0.944724
400,0.004000,0.111420,0.970588,0.970917,0.970588,0.970655


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


***** test metrics *****
  eval_accuracy  = 0.9706
  eval_f1        = 0.9707
  eval_loss      = 0.1114
  eval_precision = 0.9709
  eval_recall    = 0.9706
